In [69]:
from preprocessing import calculate_append_severity_scores, encode_class_scores
import pandas as pd

In [70]:
df = pd.read_csv('datasets/violations.csv')
df = encode_class_scores(violation_df=df)
df = calculate_append_severity_scores(violation_df=df)

In [71]:
df.head()

,violationid,buildingid,registrationid,boroid,boro,housenumber,lowhousenumber,highhousenumber,streetname,streetcode,...,councildistrict,censustract,bin,bbl,nta,certifieddate,newcertifybydate,newcorrectbydate,risk_score,risk_score_cat
0,10413107,53116,204446,2,BRONX,3565,3565,3565,BRUCKNER BOULEVARD,15920,...,13.0,26602.0,2046331.0,2.041780e+09,Pelham Bay-Country Club-City Island,NaN,NaN,NaN,1,1
1,10325067,70433,222588,2,BRONX,1120,1120,1120,EAST 225 STREET,28380,...,12.0,386.0,2090246.0,2.049030e+09,Eastchester-Edenwald-Baychester,NaN,NaN,NaN,1,1
2,10318007,43087,126972,1,MANHATTAN,504,504,506,WEST 167 STREET,37110,...,10.0,251.0,1062962.0,1.021230e+09,Washington Heights South,2014-09-25T00:00:00.000,NaN,NaN,0,0
3,10281624,92541,202243,2,BRONX,1023,1023,1023,LONGWOOD AVENUE,46820,...,17.0,89.0,2005744.0,2.027210e+09,Hunts Point,2014-07-03T00:00:00.000,NaN,NaN,1,1
4,10291962,150298,360978,3,BROOKLYN,837,837,837,43 STREET,8730,...,38.0,92.0,3018366.0,3.009250e+09,Sunset Park East,NaN,NaN,NaN,1,1


In [72]:
X = df.drop(columns=["risk_score", "risk_score_cat"])
y = df["risk_score_cat"]

In [73]:
X.columns

Index(['violationid', 'buildingid', 'registrationid', 'boroid', 'boro',
       'housenumber', 'lowhousenumber', 'highhousenumber', 'streetname',
       'streetcode', 'zip', 'apartment', 'story', 'block', 'lot', 'class',
       'inspectiondate', 'approveddate', 'originalcertifybydate',
       'originalcorrectbydate', 'ordernumber', 'novid', 'novdescription',
       'novissueddate', 'currentstatusid', 'currentstatus',
       'currentstatusdate', 'novtype', 'violationstatus', 'rentimpairing',
       'latitude', 'longitude', 'communityboard', 'councildistrict',
       'censustract', 'bin', 'bbl', 'nta', 'certifieddate', 'newcertifybydate',
       'newcorrectbydate'],
      dtype='object')

In [74]:
for column in df.columns:
    print(f"{column}")
    print(df[column].value_counts())
    print()

violationid
10413107    1
13928485    1
13721122    1
13546784    1
13910974    1
           ..
12023463    1
11623655    1
11647953    1
11852527    1
16174759    1
Name: violationid, Length: 99992, dtype: int64

buildingid
65537     64
119285    59
344337    57
77561     56
806375    55
          ..
705448     1
481124     1
268783     1
649132     1
679596     1
Name: buildingid, Length: 36294, dtype: int64

registrationid
0         3057
207907      64
201131      59
332011      57
201115      56
          ... 
207852       1
344078       1
404847       1
214298       1
401292       1
Name: registrationid, Length: 33770, dtype: int64

boroid
3    36232
2    31981
1    20925
4     9485
5     1369
Name: boroid, dtype: int64

boro
BROOKLYN         36232
BRONX            31981
MANHATTAN        20925
QUEENS            9485
STATEN ISLAND     1369
Name: boro, dtype: int64

housenumber
20        239
15        231
125       216
1         203
250       200
         ... 
135-24      1
111-08  

In [75]:
columns_to_drop = ["violationid", "buildingid", "registrationid", "boroid", "housenumber", "lowhousenumber", "highhousenumber", "streetname",
                   "class", "inspectiondate", "approveddate", "originalcertifybydate", "originalcorrectbydate",
                   "ordernumber", "novid", "novdescription", "novissueddate", "currentstatus", "novtype", 
                    "bin", "bbl", "nta", "certifieddate", "newcertifybydate", "newcorrectbydate", "currentstatusdate"]

In [76]:
# convert low cardinality to other
# streetcode, apartment, story,
# normalize / bucketize latitude and longtitude

In [77]:
X = X.drop(columns=columns_to_drop)
X.head()

,boro,streetcode,zip,apartment,story,block,lot,currentstatusid,violationstatus,rentimpairing,latitude,longitude,communityboard,councildistrict,censustract
0,BRONX,15920,10461.0,2D,2,4178,5,19,Close,N,40.847944,-73.827568,10.0,13.0,26602.0
1,BRONX,28380,10466.0,2F,2,4903,59,9,Close,N,40.883849,-73.848345,12.0,12.0,386.0
2,MANHATTAN,37110,10032.0,5C,5,2123,59,19,Close,N,40.839429,-73.937687,12.0,10.0,251.0
3,BRONX,46820,10459.0,NaN,5,2721,41,9,Close,N,40.816500,-73.896505,2.0,17.0,89.0
4,BROOKLYN,8730,11232.0,8,3,925,64,19,Close,N,40.644506,-73.998732,12.0,38.0,92.0


In [78]:
oe_vars = ["boro"]
ord_vars = ["violationstatus", "rentimpairing"]
scaler_vars = ["latitude", "longitude"]
pass_through_vars = list(set(list(X.columns)).difference(set(oe_vars + ord_vars + scaler_vars)))
print(pass_through_vars)

['block', 'censustract', 'zip', 'communityboard', 'story', 'streetcode', 'apartment', 'lot', 'councildistrict', 'currentstatusid']


In [79]:
X["apartment"] = X["apartment"].str.replace(r'\D', '', regex=True).replace('', 2.0).astype('float').clip(upper=10).fillna(2.0)

with pd.option_context('display.max_rows', None):
    print(X["apartment"].value_counts())

2.0     41939
1.0     13659
10.0    11356
3.0     11071
4.0      8307
5.0      6135
6.0      4245
7.0      1487
8.0       995
9.0       797
0.0         1
Name: apartment, dtype: int64


In [80]:
X["story"] = X["story"].str.replace(r'\D', '', regex=True).replace('', 1.0).astype('float').clip(upper=10).fillna('1.0')
with pd.option_context('display.max_rows', None):
    print(X["story"].value_counts())

1.0     31079
2.0     18463
3.0     14684
4.0     11485
5.0      8195
1.0      6360
6.0      5332
0.0      1801
10.0     1455
7.0       572
8.0       317
9.0       249
Name: story, dtype: int64


In [81]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import r2_score
from sklearn.svm import LinearSVC, SVC
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

preprocess = make_column_transformer((OrdinalEncoder(), ord_vars), (StandardScaler(), scaler_vars), (OneHotEncoder(), oe_vars), ("passthrough", pass_through_vars))

In [82]:
cols = X.columns[X.isna().any()].tolist()
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy="most_frequent")
X[cols] = imp.fit_transform(X[cols])

In [83]:
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

In [84]:
pipe = make_pipeline(preprocess, DecisionTreeClassifier(random_state=42))
pipe.fit(X_dev, y_dev)
pipe.score(X_test, y_test)

0.4584729236461823

In [85]:
pipe = make_pipeline(preprocess, RandomForestClassifier(random_state=42))
pipe.fit(X_dev, y_dev)
pipe.score(X_test, y_test)

0.5212260613030651

In [87]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    "learning_rate": [0.01, 0.1, 0.2],
    "n_estimators": [10, 100, 200],
    "max_depth": [None, 10, 100]
}
pipe = make_pipeline(preprocess, GridSearchCV(XGBClassifier(random_state=42), cv=5, param_grid=param_grid))
pipe.fit(X_dev, y_dev)
print(pipe.named_steps["gridsearchcv"].best_score_)
print(pipe.named_steps["gridsearchcv"].best_params_)

0.5719000650754824
{'learning_rate': 0.1, 'max_depth': None, 'n_estimators': 100}


In [88]:
classifer = pipe.named_steps["gridsearchcv"].best_estimator_
pipe = make_pipeline(preprocess, classifer)
pipe.score(X_test, y_test)

0.5703285164258213